In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from pyrthenope import *
import pickle

from multiprocessing import cpu_count
import concurrent.futures

print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')

pandas version: 1.3.1
numpy version: 1.21.1


In [2]:
np.finfo(float).eps

2.220446049250313e-16

In [3]:
# Some input values (i.e. DNNU) modify a diffrent output parameter (N_nu)
parth_inout_map = {
    'DNNU': 'N_nu',
    'ETA10': 'eta10',
    'TAU': 'tau'
}

# !!! will need to update card_mod if you change obs
# k, v of intrested obs: expermental errors 
observables = {
    'H2/H': 0.04 * 10**(-5),            # https://arxiv.org/pdf/1412.1408.pdf 1.2
    'Y_p': 0.0097,                      # https://arxiv.org/pdf/1412.1408.pdf 1.3 need to look more into plank 2018
    'Li7/H' : 0.3 * 10**(-10),           # https://arxiv.org/pdf/1412.1408.pdf 1.4
    'tau' : 0.6
}

# See https://arxiv.org/pdf/0705.0290.pdf Table I
# Modifcations to the base card to always be considered
card_mod = {
    'OUTPUT': 'F  4  3 4 6 8'
}

# params to vary over and step size, uses parth's card syntax
parameters = {
    'DNNU': 0.1, 
    'ETA10': 0.1,
    'TAU': 1.0
}

# cleans up output
obs_and_params = list(observables.keys()) + [parth_inout_map.get(k, k) for k in parameters.keys()]

# otherwise we will just find fisher
use_dali = True

save = True
save_file = 'dali_bbn.pkl'

# these are to match lesnpower output so we can just use the cobaya dali program
lo_spec_id = 'unlensed'
lo_experment = 0 

In [4]:
def dp(val):
    display(val)

def _run(param, mod):
    nope = Pyrthenope(card_mod=mod)
    for (k, v) in param:
        nope.card[k] = nope.card[k] + v
    return nope.run()    

def runner(obs, params, cardMods=None, doDali=True, numThreads=cpu_count()):
    # This generates all of our runs, these will run automagically and will only halt when a corresponding .result() is called
    processes = np.empty((2**doDali, len(params), len(params), 2**(1+doDali)), dtype=object)
    with concurrent.futures.ThreadPoolExecutor(max_workers=numThreads) as exec:
        for i, (param1, step1) in enumerate(params.items()):
            processes[0, i, 0, 0] = exec.submit(_run, [(param1, -step1/2)], cardMods)
            processes[0, i, 0, 1] = exec.submit(_run, [(param1,  step1/2)], cardMods)
            if doDali:
                for j, (param2, step2) in enumerate(params.items()):
                    processes[1, i, j, 0] = exec.submit(_run, [(param1, -step1/2), (param2, -step2/2)], cardMods)
                    processes[1, i, j, 1] = exec.submit(_run, [(param1,  step1/2), (param2, -step2/2)], cardMods)
                    processes[1, i, j, 2] = exec.submit(_run, [(param1, -step1/2), (param2,  step2/2)], cardMods)
                    processes[1, i, j, 3] = exec.submit(_run, [(param1,  step1/2), (param2,  step2/2)], cardMods)

    # derivative vectors
    d1Vec = pd.DataFrame()
    d2Vec = pd.DataFrame()

    #calculate the derivatives
    for i, (param1, step1) in enumerate(params.items()):
        v01 = processes[0, i, 0, 0].result()
        v02 = processes[0, i, 0, 1].result()
        
        d1 = pd.DataFrame((v02[obs] - v01[obs]) / step1).rename(index={0:param1})    
        d1Vec = pd.concat([d1Vec, d1])

        dp(f'{param1}+{step1}')
        dp(pd.concat([v01, v02.rename(index={0:1})]))

        if doDali:
            for j, (param2, step2) in enumerate(params.items()):
                v11 = processes[1, i, j, 0].result()
                v12 = processes[1, i, j, 1].result()
                v13 = processes[1, i, j, 2].result()
                v14 = processes[1, i, j, 3].result()

                d11 = (v12[obs] - v11[obs]) / step1
                d12 = (v14[obs] - v13[obs]) / step1
                d2 = pd.DataFrame((d12 - d11) / step2).rename(index={0:(param1, param2)})
                d2Vec = pd.concat([d2Vec, d2])

                dp(f'{param1}+{step1}, {param2}+{step2}')
                dp(pd.concat([v11, v12.rename(index={0:1}), v13.rename(index={0:2}), v14.rename(index={0:3})]))

    # Finds the (gaussian) 1/sigma^2 errors
    errors = np.array([list(obs.values())])
    isigma2 = pd.DataFrame(
        np.linalg.inv(errors.T * np.identity(len(list(obs))) * errors), 
        index=[list(obs.keys())], columns=[list(obs.keys())])

    dp('First Derivatives')
    dp(d1Vec)
    dp('Second Derivatives')
    dp(d2Vec)
    dp('Inverse Sigma**2')
    dp(isigma2)

    # calculate fisher, dali3 and dali4
    # https://arxiv.org/pdf/1401.6892.pdf 15
    fisher = np.einsum('ia,ab,jb', d1Vec, isigma2, d1Vec)
    if doDali:
        # generate tensors so we can use these in np.einsum
        d2Ten = d2Vec.values.reshape(len(params), len(params), len(obs))

        dali3 = np.einsum('ija,ab,kb', d2Ten, isigma2, d1Vec)
        dali4 = np.einsum('ija,ab,klb', d2Ten, isigma2, d2Ten)

        return fisher, dali3, dali4

    return fisher, None, None


In [5]:
baseData = _run([], card_mod)
dp(baseData)

,N_nu,xie,xix,tau,rholmbd,eta10,OmegaBh^2,phie,thetah,tnuh,nbh,N/H,Y_H,H2/H,H3/H,He3/H,Y_p,Li6/H,Li7/H,Be7/H
0,3.0,0.0,0.0,879.4,0.0,6.13832,0.02242,39.4153,8.685500e-26,0.714958,5.096670e-16,9.169530e-10,0.754432,0.000025,8.064730e-08,0.00001,0.246878,1.090290e-14,4.687700e-10,4.416040e-10


In [6]:
fisher, dali3, dali4 = runner(observables, parameters, card_mod, doDali=use_dali)
dp(f'fisher {fisher}')
dp(f'dali3 {dali3}')
dp(f'dali4 {dali4}')

if save:
    fid_values = {
        'N_nu': baseData['N_nu'].values[0], #3.0,
        # 'omega_b_h2': baseData['OmegaBh^2'].values[0], #0.02242 # check that this corresponds to our fid eta10 6.13332, is this needed?
        'eta10': baseData['eta10'].values[0],
        'tau': baseData['tau'].values[0]
    }
    saveData = {'cosmoFid': fid_values, 'fisherGaussian': {lo_experment: {lo_spec_id: fisher}}}
    if use_dali:
        saveData['DALI3Gaussian'] =  {lo_experment: {lo_spec_id: dali3}}
        saveData['DALI4Gaussian'] =  {lo_experment: {lo_spec_id: dali4}}

    with open(save_file, 'wb') as file:
        pickle.dump(saveData, file, protocol=pickle.HIGHEST_PROTOCOL)
        
    

'DNNU+0.1'

,N_nu,xie,xix,tau,rholmbd,eta10,OmegaBh^2,phie,thetah,tnuh,nbh,N/H,Y_H,H2/H,H3/H,He3/H,Y_p,Li6/H,Li7/H,Be7/H
0,2.95,0.0,0.0,879.4,0.0,6.13832,0.02242,39.4157,8.656080e-26,0.714958,5.096640e-16,9.047470e-10,0.755107,0.000025,8.006500e-08,0.00001,0.246205,1.079040e-14,4.709550e-10,4.440380e-10
1,3.05,0.0,0.0,879.4,0.0,6.13832,0.02242,39.4149,8.714810e-26,0.714958,5.096640e-16,9.292040e-10,0.753768,0.000025,8.122750e-08,0.00001,0.247548,1.101510e-14,4.665920e-10,4.391770e-10


'DNNU+0.1, DNNU+0.1'

,N_nu,xie,xix,tau,rholmbd,eta10,OmegaBh^2,phie,thetah,tnuh,nbh,N/H,Y_H,H2/H,H3/H,He3/H,Y_p,Li6/H,Li7/H,Be7/H
0,2.9,0.0,0.0,879.4,0.0,6.13832,0.02242,39.4161,8.626560e-26,0.714958,5.096700e-16,8.926460e-10,0.755785,0.000025,7.948450e-08,0.00001,0.245522,1.067850e-14,4.731470e-10,4.464780e-10
1,3.0,0.0,0.0,879.4,0.0,6.13832,0.02242,39.4153,8.685500e-26,0.714958,5.096670e-16,9.169530e-10,0.754432,0.000025,8.064730e-08,0.00001,0.246878,1.090290e-14,4.687700e-10,4.416040e-10
2,3.0,0.0,0.0,879.4,0.0,6.13832,0.02242,39.4153,8.685500e-26,0.714958,5.096670e-16,9.169530e-10,0.754432,0.000025,8.064730e-08,0.00001,0.246878,1.090290e-14,4.687700e-10,4.416040e-10
3,3.1,0.0,0.0,879.4,0.0,6.13832,0.02242,39.4146,8.744030e-26,0.714958,5.096710e-16,9.415540e-10,0.753103,0.000025,8.180920e-08,0.00001,0.248211,1.112800e-14,4.644300e-10,4.367660e-10


'DNNU+0.1, ETA10+0.1'

...

'Inverse Sigma**2'

,H2/H,Y_p,Li7/H,tau
H2/H,6.250000e+12,0.000000,0.000000e+00,0.000000
Y_p,0.000000e+00,10628.122011,0.000000e+00,0.000000
Li7/H,0.000000e+00,0.000000,1.111111e+21,0.000000
tau,0.000000e+00,0.000000,0.000000e+00,2.777778


'fisher [[ 7.63670506e+01 -1.49335731e+02  2.75715729e-01]\n [-1.49335731e+02  3.06143937e+02 -4.59946253e-01]\n [ 2.75715729e-01 -4.59946253e-01  2.77921845e+00]]'

'dali3 [[[-1.99603484e+00  3.53677328e+00 -9.43657714e-03]\n  [-1.57132347e+01  3.00105954e+01 -6.01776833e-02]\n  [-6.19010212e-01  1.30863158e+00 -1.67189671e-03]]\n\n [[-1.57132347e+01  3.00105954e+01 -6.01776833e-02]\n  [ 5.29853474e+01 -1.01742235e+02  2.00172611e-01]\n  [-2.60293481e-02  4.30107022e-02 -1.39652214e-04]]\n\n [[-6.19010212e-01  1.30863158e+00 -1.67189671e-03]\n  [-2.60293481e-02  4.30107022e-02 -1.39652214e-04]\n  [-4.76687197e-03  6.47126107e-03 -3.32850536e-05]]]'

'dali4 [[[[ 1.02058877e-01  2.95888913e-01  1.07930154e-02]\n   [ 2.95888913e-01 -1.05679758e+00  1.21195153e-03]\n   [ 1.07930154e-02  1.21195153e-03  2.17501653e-04]]\n\n  [[ 2.95888913e-01  3.71038406e+00  1.31763766e-01]\n   [ 3.71038406e+00 -1.22418164e+01  4.67632881e-03]\n   [ 1.31763766e-01  4.67632881e-03  1.06299524e-03]]\n\n  [[ 1.07930154e-02  1.31763766e-01  5.89951384e-03]\n   [ 1.31763766e-01 -4.42957884e-01  1.33157689e-04]\n   [ 5.89951384e-03  1.33157689e-04  1.74914237e-05]]]\n\n\n [[[ 2.95888913e-01  3.71038406e+00  1.31763766e-01]\n   [ 3.71038406e+00 -1.22418164e+01  4.67632881e-03]\n   [ 1.31763766e-01  4.67632881e-03  1.06299524e-03]]\n\n  [[-1.05679758e+00 -1.22418164e+01 -4.42957884e-01]\n   [-1.22418164e+01  4.05242161e+01 -1.60563210e-02]\n   [-4.42957884e-01 -1.60563210e-02 -3.50853973e-03]]\n\n  [[ 1.21195153e-03  4.67632881e-03  1.33157689e-04]\n   [ 4.67632881e-03 -1.60563210e-02  1.59264209e-05]\n   [ 1.33157689e-04  1.59264209e-05  3.37079451e-06]]]\n\